# Урок 8
## Двухфакторный дисперсионный анализ. Факторный анализ. Логистическая регрессия

### Задача 1

#### Условие:
Объясняемая переменная `y` зависит от двух категориальных факторов `A` и `B`, причём фактор `A` имеет `3` уровня, а фактор `B` - `4` уровня. Результаты наблюдений приведены в таблице:

```json
y = [[2.68, 3.29, 2.88, 4.45],
     [4.12, 4.96, 5.09, 5.22],
     [5.52, 4.50, 5.42, 5.29]]
```
С помощью двухфакторного дисперсионного анализа проверьте влияние каждого из факторов на переменную `y`.

#### Решение:

In [1]:
import numpy as np

In [2]:
y = np.array([[2.68, 3.29, 2.88, 4.45],
              [4.12, 4.96, 5.09, 5.22],
              [5.52, 4.50, 5.42, 5.29]])

Найдем массив средних значений для факторов `A` и `B`:

In [3]:
a_means = np.mean(y, axis=1, keepdims=False)
a_means 

array([3.325 , 4.8475, 5.1825])

In [4]:
b_means = np.mean(y, axis=0, keepdims=False)
b_means 

array([4.10666667, 4.25      , 4.46333333, 4.98666667])

Также найдем среднее значение `y`:

In [5]:
y_mean = y.mean()
y_mean

4.451666666666666

Найдем суммы квадратов отклонений:
$$SS_a = k \cdot \displaystyle\sum_{i=1}^m \left( \overline{Y_{i \ast}} - \overline{Y} \right)^2, \:\:
SS_b = m \cdot \displaystyle\sum_{j=1}^k \left( \overline{Y_{\ast j}} - \overline{Y} \right)^2, \:\:
SS_w = \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k \left( y_{ij} - \overline{Y_{i \ast}} - \overline{Y_{\ast j}} + \overline{Y} \right)^2$$

В нашем случае:

$k = 4$

$m = 3$

In [6]:
k = 4
m = 3

In [7]:
SS_a = k * sum((a_means - y_mean) ** 2)
SS_a

7.8407166666666654

In [8]:
SS_b = m * sum((b_means - y_mean) ** 2)
SS_b

1.338166666666669

In [9]:
SS_w = 0
for i in range(m):
    for j in range(k):
        SS_w += (y[i][j] - a_means[i] - b_means[j] + y_mean) ** 2
SS_w

1.9298833333333327

Оценки дисперсий:
$$\sigma_a^2 = \dfrac{SS_a}{m - 1}, \:\: \sigma_b^2 = \dfrac{SS_b}{k - 1}, \:\:
\sigma_w^2 = \dfrac{SS_w}{(k - 1) (m - 1)}$$

In [10]:
sigma_a = SS_a / (m - 1)
sigma_b = SS_b / (k - 1)
sigma_w = SS_w / ((k - 1) * (m - 1))

sigma_a, sigma_b, sigma_w 

(3.9203583333333327, 0.4460555555555563, 0.32164722222222214)

Найдем значение статистик:
$$F_a = \dfrac{\sigma_a^2}{\sigma_w^2}$$

В предположении верности гипотезы $H_{0a}$ эта статистика имеет распределение Фишера с параметрами $k_{1a} = m - 1$, $k_{2a} = n - m$, где $n = m \cdot k$ — общее число элементов в выборке. Далее, как обычно, строится критическая область (правосторонняя, поскольку распределение Фишера имеет один хвост), и проводится тест.

Аналогично, для гипотезы о влиянии фактора $b$ статистика:
$$F_b = \dfrac{\sigma_b^2}{\sigma_w^2}$$

In [11]:
F_a = sigma_a / sigma_w
F_b = sigma_b / sigma_w
F_a, F_b

(12.188379262995173, 1.386785038819275)

Найдем критические области при $\alpha = 0.05$:

In [12]:
from scipy import stats

alpha = 0.05

Для фактора `A`:

In [13]:
t = stats.f.ppf(1 - alpha, 2, 9)
t

4.25649472909375

Критическая область:
$$\Omega_{a\alpha} = (4.256495, +\infty)$$

In [14]:
F_a > t

True

Для фактора `B`:

In [15]:
t = stats.f.ppf(1 - alpha, 3, 8)
t

4.06618055135116

Критическая область:
$$\Omega_{b\alpha} = (4.066181, +\infty)$$

In [16]:
F_b > t

False

Получаем, что фактор `A` влияет на значение переменной `y`, а фактор `B` - нет.